In [1]:
import sys
sys.path.insert(0, "/home/pwahle/proliferating_RGC/")

import yaml
import importlib
import modules
import utils
importlib.reload(modules)
import os
from pathlib import Path
import subprocess

# load global variables and parameters
with open("../params.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

globals().update(cfg)

from scipy import ndimage
import re
import cv2
from scipy import ndimage as ndi
import copy
import numpy as np
import matplotlib as mpl
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from bs4 import BeautifulSoup




In [58]:
path = data_path + 'exp5/'

outname = 'multipoints_cycle2_updated.xml'
origina_file = 'multipoints_original.xml'
new_file = 'multipoints_cycle2_measured.xml'


with open(path + origina_file, 'rb') as f:
    data = f.read()
Bs_orig = BeautifulSoup(data, "xml")

with open(path + new_file, 'rb') as f:
    data = f.read()
Bs_new = BeautifulSoup(data, "xml")


a1 = np.array([float(Bs_orig.find_all('dXPosition', {'runtype':'double'})[0]['value']),float(Bs_orig.find_all('dYPosition', {'runtype':'double'})[0]['value']),float(Bs_orig.find_all('dZPosition', {'runtype':'double'})[0]['value'])])
a2 = np.array([float(Bs_orig.find_all('dXPosition', {'runtype':'double'})[1]['value']),float(Bs_orig.find_all('dYPosition', {'runtype':'double'})[1]['value']),float(Bs_orig.find_all('dZPosition', {'runtype':'double'})[1]['value'])])
a3 = np.array([float(Bs_orig.find_all('dXPosition', {'runtype':'double'})[2]['value']),float(Bs_orig.find_all('dYPosition', {'runtype':'double'})[2]['value']),float(Bs_orig.find_all('dZPosition', {'runtype':'double'})[2]['value'])])
v0 = np.array([a1,a2,a3])

b1 = np.array([float(Bs_new.find_all('dXPosition', {'runtype':'double'})[0]['value']),float(Bs_new.find_all('dYPosition', {'runtype':'double'})[0]['value']),float(Bs_new.find_all('dZPosition', {'runtype':'double'})[0]['value'])])
b2 = np.array([float(Bs_new.find_all('dXPosition', {'runtype':'double'})[1]['value']),float(Bs_new.find_all('dYPosition', {'runtype':'double'})[1]['value']),float(Bs_new.find_all('dZPosition', {'runtype':'double'})[1]['value'])])
b3 = np.array([float(Bs_new.find_all('dXPosition', {'runtype':'double'})[2]['value']),float(Bs_new.find_all('dYPosition', {'runtype':'double'})[2]['value']),float(Bs_new.find_all('dZPosition', {'runtype':'double'})[2]['value'])])
v1 = np.array([b1,b2,b3])

T = utils.affine_matrix_from_points(v0, v1,shear=False, scale=False, usesvd=True)

bX = []
for i in Bs_orig.find_all('dXPosition', {'runtype':'double'})[3:]:
    bX.append(float(i['value']))
bY = []
for i in Bs_orig.find_all('dYPosition', {'runtype':'double'})[3:]:
    bY.append(float(i['value']))
bZ = []
for i in Bs_orig.find_all('dZPosition', {'runtype':'double'})[3:]:
    bZ.append(float(i['value']))
v3 = np.squeeze(np.dstack([bX,bY,bZ,np.ones(len(bZ))]))

projected_points = np.matmul(v3,T)

for tag in np.arange(len(Bs_orig.find_all('dXPosition', {'runtype':'double'})))[3:]:
    Bs_orig.find_all('dXPosition', {'runtype':'double'})[tag]['value'] = str(projected_points[tag-3,0])
    
for tag in np.arange(len(Bs_orig.find_all('dXPosition', {'runtype':'double'})))[:3]:
    Bs_orig.find_all('dXPosition', {'runtype':'double'})[tag]['value'] = str(v1[tag,0])
    
for tag in np.arange(len(Bs_orig.find_all('dYPosition', {'runtype':'double'})))[3:]:
    Bs_orig.find_all('dYPosition', {'runtype':'double'})[tag]['value'] = str(projected_points[tag-3,1])
    
for tag in np.arange(len(Bs_orig.find_all('dYPosition', {'runtype':'double'})))[:3]:
    Bs_orig.find_all('dYPosition', {'runtype':'double'})[tag]['value'] = str(v1[tag,1])
    
for tag in np.arange(len(Bs_orig.find_all('dZPosition', {'runtype':'double'})))[3:]:
    Bs_orig.find_all('dZPosition', {'runtype':'double'})[tag]['value'] = str(projected_points[tag-3,2])
    
for tag in np.arange(len(Bs_orig.find_all('dZPosition', {'runtype':'double'})))[:3]:
    Bs_orig.find_all('dZPosition', {'runtype':'double'})[tag]['value'] = str(v1[tag,2])
    
    
with open(path + outname, "w") as f:
    f.write(Bs_orig.prettify())
    
print('updated points in file ' + outname)

updated points in file multipoints_cycle2_updated.xml


In [27]:
xx, yy, zz = np.sum(v0 * v1, axis=1)